Reference: https://github.com/dh1105/Sentence-Entailment

In [49]:
# from google.colab import drive
# drive.mount('/content/drive')

In [50]:
# !pip install transformers
# !pip install sentencepiece
## !pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.7-cp36-cp36m-linux_x86_64.whl

In [1]:
import pandas as pd
import re
import torch
# import torch_xla
# import torch_xla.core.xla_model as xm
from torch.utils.data import Dataset, TensorDataset, DataLoader, SequentialSampler, RandomSampler
from torch.nn.utils.rnn import pad_sequence
# from keras.preprocessing.sequence import pad_sequences
import pickle
import os
import numpy as np

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
task_a_hyp = "This is offensive speech."
task_b_hyp = "This is targeted offense."
task_c_hyp_1= "This is targeted towards an individual."
task_c_hyp_2 = "This is targeted towards a group."
task_c_hyp_3 = "This isn't targeted towards a group or an individual."
f = open('Data/OffensEval/olid_train_v2.csv')
f_a = open('Data/OffensEval/bert_nli.csv', 'w')
f_a.write('tweet_id' + '\t' + 'gold_label' + '\t' + 'sentence1' + '\t' + 'sentence2' + '\n')
lines = f.readlines()
for line in lines[1:]:
  row = line.split('\t')
  tweet_id = row[0].strip()
  tweet_text = ' '.join(row[1:-3]).strip()
  is_offensive = row[-3].strip()
  is_targeted = row[-2].strip()
  target = row[-1].strip()
  if is_offensive == 'OFF':
    f_a.write(tweet_id + '\t' + 'entailment' + '\t' + tweet_text + '\t' + task_a_hyp + '\n')
    if is_targeted == 'TIN':
      f_a.write(tweet_id + '\t' + 'entailment' + '\t' + tweet_text + '\t' + task_b_hyp + '\n')
      if target == 'IND':
        f_a.write(tweet_id + '\t' + 'entailment' + '\t' + tweet_text + '\t' + task_c_hyp_1 + '\n')
        f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_c_hyp_2 + '\n')
        f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_c_hyp_3 + '\n')
      elif target == 'GRP':
        f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_c_hyp_1 + '\n')
        f_a.write(tweet_id + '\t' + 'entailment' + '\t' + tweet_text + '\t' + task_c_hyp_2 + '\n')
        f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_c_hyp_3 + '\n')
      elif target == 'OTH':
        f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_c_hyp_1 + '\n')
        f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_c_hyp_2 + '\n')
        f_a.write(tweet_id + '\t' + 'entailment' + '\t' + tweet_text + '\t' + task_c_hyp_3 + '\n')
    elif is_targeted == 'UNT':
      f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_b_hyp + '\n')
  else:
    f_a.write(tweet_id + '\t' + 'contradiction' + '\t' + tweet_text + '\t' + task_a_hyp + '\n')

In [4]:
df = pd.read_csv('./Data/OffensEval/bert_nli.csv', delimiter='\t')

In [5]:
df = df.sample(frac=1)

In [6]:
l = len(df)
train_df = df[:int(0.8*l)]
val_df = df[int(0.8*l):int(0.9*l)]
test_df = df[int(0.9*l):]

In [7]:
train_df = train_df.dropna()
val_df = val_df.dropna()
test_df = test_df.dropna()

In [8]:
train_df['sentence1'] = train_df['sentence1'].astype(str)
train_df['sentence2'] = train_df['sentence2'].astype(str)

In [9]:
val_df['sentence1'] = val_df['sentence1'].astype(str)
val_df['sentence2'] = val_df['sentence2'].astype(str)

In [10]:
test_df['sentence1'] = test_df['sentence1'].astype(str)
test_df['sentence2'] = test_df['sentence2'].astype(str)

In [11]:
train_df = train_df[(train_df['sentence1'].str.split().str.len() > 0) & (train_df['sentence2'].str.split().str.len() > 0)]
val_df = val_df[(val_df['sentence1'].str.split().str.len() > 0) & (val_df['sentence2'].str.split().str.len() > 0)]
test_df = test_df[(test_df['sentence1'].str.split().str.len() > 0) & (test_df['sentence2'].str.split().str.len() > 0)]

In [12]:
train_df

,tweet_id,gold_label,sentence1,sentence2
18190,31639,entailment,According to a student that spoke under condit...,This is offensive speech.
27505,74555,entailment,@USER It's really slowing me down. I wanna ask...,This is offensive speech.
5453,20777,contradiction,@USER @USER @USER @USER @USER Trump publicly l...,This isn't targeted towards a group or an indi...
9458,75929,entailment,@USER Remind me of Antifa ratbags.,This is targeted offense.
19799,42320,entailment,@USER @USER @USER @USER Bridget that's another...,This is targeted offense.
...,...,...,...,...
12421,11581,entailment,@USER @USER @USER @USER @USER This is the viol...,This is targeted towards a group.
25556,81405,entailment,@USER He also said he had a gatlin gun and nig...,This is targeted offense.
12669,80291,entailment,@USER Fairness?? What about being fair to the ...,This is targeted offense.
22893,42537,entailment,Mueller proves the judicial system is weaponiz...,This is offensive speech.


In [13]:
val_df

,tweet_id,gold_label,sentence1,sentence2
289,28601,contradiction,@USER @USER @USER Why can't liberals read?????...,This is offensive speech.
5587,54841,entailment,"@USER Being the good Christian that he is, he ...",This is offensive speech.
27185,92451,contradiction,.@USER @USER and @USER MP @USER praises the 'i...,This is offensive speech.
760,61093,contradiction,@USER @USER @USER @USER @USER @USER @USER @USE...,This is offensive speech.
27082,48637,entailment,@USER What happens next is whoever inthe GOP v...,This is targeted offense.
...,...,...,...,...
2949,69593,contradiction,@USER @USER @USER @USER @USER @USER @USER @USE...,This is targeted towards a group.
2312,10755,contradiction,@USER You are right Joe. I can do an interview...,This is offensive speech.
10139,88558,entailment,@USER @USER I’m sure this pope is on a first n...,This is targeted towards an individual.
9416,73895,contradiction,IT'S TRUE! #MAGA #NoCollusion URL,This is offensive speech.


In [14]:
test_df

,tweet_id,gold_label,sentence1,sentence2
475,65785,contradiction,@USER Always thought H Ford was a level headed...,This is offensive speech.
5445,26623,entailment,@USER @USER @USER So many issues facing Austra...,This is targeted offense.
26548,52777,contradiction,@USER @USER John. You are winning them over o...,This is offensive speech.
13950,37068,contradiction,@USER He is that stupid!,This isn't targeted towards a group or an indi...
20376,19289,entailment,@USER She is a complete idiot,This is targeted towards an individual.
...,...,...,...,...
9305,48788,entailment,@USER Fucking sucks when the refs are against ...,This is targeted towards an individual.
15725,58638,contradiction,@USER @USER @USER @USER @USER The idea that ca...,This is offensive speech.
5257,10576,contradiction,@USER this is rei from evangelion concept/desi...,This is offensive speech.
16942,19659,entailment,@USER @USER Dolt. You’ve become another nail f...,This is targeted offense.


In [15]:
test_df_a = test_df[test_df.sentence2 == "This is offensive speech."].sort_values(by=['tweet_id'])
test_df_b = test_df[test_df.sentence2 == "This is targeted offense."].sort_values(by=['tweet_id'])
test_df_c = test_df[test_df.sentence2 == "This is targeted towards an individual."].sort_values(by=['tweet_id'])
test_df_d = test_df[test_df.sentence2 == "This is targeted towards a group."].sort_values(by=['tweet_id'])
test_df_e = test_df[test_df.sentence2 == "This isn't targeted towards a group or an individual."].sort_values(by=['tweet_id'])

In [16]:
print(len(test_df_a))
print(len(test_df_b))
print(len(test_df_c))
print(len(test_df_d))
print(len(test_df_e))

1262
431
398
363
362


In [17]:
import torch
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import pickle
import os
from transformers import BertTokenizer

class MNLIDataBert(Dataset):

  def __init__(self, train_df, val_df, test_df, test_df_a, test_df_b, test_df_c, test_df_d, test_df_e):
    self.label_dict = {'entailment': 1, 'contradiction': 0}

    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df
    self.test_df_a = test_df_a
    self.test_df_b = test_df_b
    self.test_df_c = test_df_c
    self.test_df_d = test_df_d
    self.test_df_e = test_df_e
    self.base_path = 'mnli-data-offenseval'
    self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
    self.train_data = None
    self.val_data = None
    self.test_data = None
    self.test_data_a = None
    self.test_data_b = None
    self.test_data_c = None
    self.test_data_d = None
    self.test_data_e = None
    self.init_data()

  def init_data(self):
    #Saving takes too much RAM
    
    if os.path.exists(os.path.join(self.base_path, 'train_data.pkl')):
      print("Found training data")
      with open(os.path.join(self.base_path, 'train_data.pkl'), 'rb') as f:
        self.train_data = pickle.load(f)
    else:
      self.train_data = self.load_data(self.train_df)
      with open(os.path.join(self.base_path, 'train_data.pkl'), 'wb') as f:
        pickle.dump(self.train_data, f)
    if os.path.exists(os.path.join(self.base_path, 'val_data.pkl')):
      print("Found val data")
      with open(os.path.join(self.base_path, 'val_data.pkl'), 'rb') as f:
        self.val_data = pickle.load(f)
    else:
      self.val_data = self.load_data(self.val_df)
      with open(os.path.join(self.base_path, 'val_data.pkl'), 'wb') as f:
        pickle.dump(self.val_data, f)
    if os.path.exists(os.path.join(self.base_path, 'test_data.pkl')):
      print("Found test data")
      with open(os.path.join(self.base_path, 'test_data.pkl'), 'rb') as f:
        self.test_data = pickle.load(f)
    else:
      self.test_data = self.load_data(self.test_df)
      with open(os.path.join(self.base_path, 'test_data.pkl'), 'wb') as f:
        pickle.dump(self.test_data, f)
    if os.path.exists(os.path.join(self.base_path, 'test_data_a.pkl')):
      print("Found test data a")
      with open(os.path.join(self.base_path, 'test_data_a.pkl'), 'rb') as f:
        self.test_data_a = pickle.load(f)
    else:
      self.test_data_a = self.load_data(self.test_df_a)
      with open(os.path.join(self.base_path, 'test_data_a.pkl'), 'wb') as f:
        pickle.dump(self.test_data_a, f)
    if os.path.exists(os.path.join(self.base_path, 'test_data_b.pkl')):
      print("Found test data b")
      with open(os.path.join(self.base_path, 'test_data_b.pkl'), 'rb') as f:
        self.test_data_b = pickle.load(f)
    else:
      self.test_data_b = self.load_data(self.test_df_b)
      with open(os.path.join(self.base_path, 'test_data_b.pkl'), 'wb') as f:
        pickle.dump(self.test_data_b, f)
    if os.path.exists(os.path.join(self.base_path, 'test_data_c.pkl')):
      print("Found test data c")
      with open(os.path.join(self.base_path, 'test_data_c.pkl'), 'rb') as f:
        self.test_data_c = pickle.load(f)
    else:
      self.test_data_c = self.load_data(self.test_df_c)
      with open(os.path.join(self.base_path, 'test_data_c.pkl'), 'wb') as f:
        pickle.dump(self.test_data_c, f)
    if os.path.exists(os.path.join(self.base_path, 'test_data_d.pkl')):
      print("Found test data d")
      with open(os.path.join(self.base_path, 'test_data_d.pkl'), 'rb') as f:
        self.test_data_d = pickle.load(f)
    else:
      self.test_data_d = self.load_data(self.test_df_d)
      with open(os.path.join(self.base_path, 'test_data_d.pkl'), 'wb') as f:
        pickle.dump(self.test_data_d, f)
    if os.path.exists(os.path.join(self.base_path, 'test_data_e.pkl')):
      print("Found test data e")
      with open(os.path.join(self.base_path, 'test_data_e.pkl'), 'rb') as f:
        self.test_data_e = pickle.load(f)
    else:
      self.test_data_e = self.load_data(self.test_df_e)
      with open(os.path.join(self.base_path, 'test_data_e.pkl'), 'wb') as f:
        pickle.dump(self.test_data_e, f)
    # self.train_data = self.load_data(self.train_df)
    # self.val_data = self.load_data(self.val_df)
    # self.test_data = self.load_data(self.test_df)
  def load_data(self, df):
    MAX_LEN = 512
    token_ids = []
    mask_ids = []
    seg_ids = []
    y = []

    premise_list = df['sentence1'].to_list()
    hypothesis_list = df['sentence2'].to_list()
    label_list = df['gold_label'].to_list()

    for (premise, hypothesis, label) in zip(premise_list, hypothesis_list, label_list):
      premise_id = self.tokenizer.encode(premise, add_special_tokens = False)
      hypothesis_id = self.tokenizer.encode(hypothesis, add_special_tokens = False)
      pair_token_ids = [self.tokenizer.cls_token_id] + premise_id + [self.tokenizer.sep_token_id] + hypothesis_id + [self.tokenizer.sep_token_id]
      premise_len = len(premise_id)
      hypothesis_len = len(hypothesis_id)

      segment_ids = torch.tensor([0] * (premise_len + 2) + [1] * (hypothesis_len + 1))  # sentence 0 and sentence 1
      attention_mask_ids = torch.tensor([1] * (premise_len + hypothesis_len + 3))  # mask padded values

      token_ids.append(torch.tensor(pair_token_ids))
      seg_ids.append(segment_ids)
      mask_ids.append(attention_mask_ids)
      y.append(self.label_dict[label])
    
    token_ids = pad_sequence(token_ids, batch_first=True)
    mask_ids = pad_sequence(mask_ids, batch_first=True)
    seg_ids = pad_sequence(seg_ids, batch_first=True)
    y = torch.tensor(y)
    dataset = TensorDataset(token_ids, mask_ids, seg_ids, y)
    print(len(dataset))
    return dataset

  def get_data_loaders(self, batch_size=32, shuffle=True):
    train_loader = DataLoader(
      self.train_data,
      shuffle=shuffle,
      batch_size=batch_size
    )

    val_loader = DataLoader(
      self.val_data,
      shuffle=shuffle,
      batch_size=batch_size
    )

    test_loader = DataLoader(
      self.test_data,
      shuffle=shuffle,
      batch_size=batch_size
    )

    test_loader_a = DataLoader(
      self.test_data_a,
      shuffle=shuffle,
      batch_size=batch_size
    )

    test_loader_b = DataLoader(
      self.test_data_b,
      shuffle=shuffle,
      batch_size=batch_size
    )

    test_loader_c = DataLoader(
      self.test_data_c,
      shuffle=shuffle,
      batch_size=batch_size
    )

    test_loader_d = DataLoader(
      self.test_data_d,
      shuffle=shuffle,
      batch_size=batch_size
    )

    test_loader_e = DataLoader(
      self.test_data_e,
      shuffle=shuffle,
      batch_size=batch_size
    )

    return train_loader, val_loader, test_loader, test_loader_a, test_loader_b, test_loader_c, test_loader_d, test_loader_e

In [18]:
mnli_dataset = MNLIDataBert(train_df, val_df, test_df, test_df_a, test_df_b, test_df_c, test_df_d, test_df_e)

22520
2815
2816
1262
431
398
363
362


In [19]:
train_loader, val_loader, test_loader, test_loader_a, test_loader_b, test_loader_c, test_loader_d, test_loader_e = mnli_dataset.get_data_loaders(batch_size=64)

In [20]:
from transformers import BertForSequenceClassification, AdamW

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [21]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

In [22]:
# This variable contains all of the hyperparemeter information our training loop needs
optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5, correct_bias=False)

In [23]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 109,483,778 trainable parameters


In [24]:
def multi_acc(y_pred, y_test):
  acc = (torch.log_softmax(y_pred, dim=1).argmax(dim=1) == y_test).sum().float() / float(y_test.size(0))
  return acc

In [26]:
import time

EPOCHS = 5

def train(model, train_loader, val_loader, optimizer):  
  total_step = len(train_loader)
  #model.load_state_dict(torch.load('/content/drive/MyDrive/COMPSCI 685 Advanced Natural Language Processing/Project/Weights/nli_0.pt'))
  #model.load_state_dict(torch.load('./Weights/nli_3.pt'))
  for epoch in range(EPOCHS):
    start = time.time()
    model.train()
    total_train_loss = 0
    total_train_acc  = 0
    for batch_idx, (pair_token_ids, mask_ids, seg_ids, y) in enumerate(train_loader):
      optimizer.zero_grad()
      if batch_idx%100 == 0:
          print(batch_idx, len(train_loader))
      pair_token_ids = pair_token_ids.to(device)
      mask_ids = mask_ids.to(device)
      seg_ids = seg_ids.to(device)
      labels = y.to(device)
      # prediction = model(pair_token_ids, mask_ids, seg_ids)
      loss, prediction = model(pair_token_ids, 
                             token_type_ids=seg_ids, 
                             attention_mask=mask_ids, 
                             labels=labels).values()

      # loss = criterion(prediction, labels)
      acc = multi_acc(prediction, labels)

      loss.backward()
      optimizer.step()
      
      total_train_loss += loss.item()
      total_train_acc  += acc.item()

    train_acc  = total_train_acc/len(train_loader)
    train_loss = total_train_loss/len(train_loader)
    model.eval()
    total_val_acc  = 0
    total_val_loss = 0
    with torch.no_grad():
      for batch_idx, (pair_token_ids, mask_ids, seg_ids, y) in enumerate(val_loader):
        optimizer.zero_grad()
        pair_token_ids = pair_token_ids.to(device)
        mask_ids = mask_ids.to(device)
        seg_ids = seg_ids.to(device)
        labels = y.to(device)

        # prediction = model(pair_token_ids, mask_ids, seg_ids)
        loss, prediction = model(pair_token_ids, 
                             token_type_ids=seg_ids, 
                             attention_mask=mask_ids, 
                             labels=labels).values()
        
        # loss = criterion(prediction, labels)
        acc = multi_acc(prediction, labels)

        total_val_loss += loss.item()
        total_val_acc  += acc.item()

    val_acc  = total_val_acc/len(val_loader)
    val_loss = total_val_loss/len(val_loader)
    end = time.time()
    hours, rem = divmod(end-start, 3600)
    minutes, seconds = divmod(rem, 60)
    #torch.save(model.state_dict(),os.path.join('/content/drive/MyDrive/COMPSCI 685 Advanced Natural Language Processing/Project/Weights', 'nli_{}.pt'.format(epoch)))
    torch.save(model.state_dict(),os.path.join('./WeightsOffenseval', 'nli_latest_{}.pt'.format(epoch)))
    print(f'Epoch {epoch+1}: train_loss: {train_loss:.4f} train_acc: {train_acc:.4f} | val_loss: {val_loss:.4f} val_acc: {val_acc:.4f}')
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

In [43]:
import time
import math
def test(model, test_loader):
  model.load_state_dict(torch.load('WeightsOffenseval/nli_latest_3.pt', map_location=torch.device('cpu')))
  #model.load_state_dict(torch.load('WeightsHateval/nli_new_4.pt', map_location=torch.device('cpu')))
  model.eval()
  total_test_acc  = 0
  total_test_loss = 0
  start = time.time()
  preds = []
  acts = []
  with torch.no_grad():
    #print("Prediction", "Label")
    for batch_idx, (pair_token_ids, mask_ids, seg_ids, y) in enumerate(test_loader):
      optimizer.zero_grad()
      pair_token_ids = pair_token_ids.to(device)
      mask_ids = mask_ids.to(device)
      seg_ids = seg_ids.to(device)
      labels = y.to(device)

      # prediction = model(pair_token_ids, mask_ids, seg_ids)
      loss, prediction = model(pair_token_ids, 
                           token_type_ids=seg_ids, 
                           attention_mask=mask_ids, 
                           labels=labels).values()
        
      # loss = criterion(prediction, labels)
      acc = multi_acc(prediction, labels)
      preds = preds + prediction.tolist()
      acts = acts + labels.tolist()
      #print(prediction, labels)
      total_test_loss += loss.item()
      total_test_acc  += acc.item() 

  test_acc  = total_test_acc/len(test_loader)
  test_loss = total_test_loss/len(test_loader)
  end = time.time()
  hours, rem = divmod(end-start, 3600)
  minutes, seconds = divmod(rem, 60)
  print(f'test_loss: {test_loss:.4f} test_acc: {test_acc:.4f}')
  print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
  #return [math.exp(preds[i][1])/(math.exp(preds[i][0] + math.exp(preds[i][1]))) for i in range(len(preds))], acts
  #return [0 if preds[i][0] > preds[i][1] else 1 for i in range(len(preds))], acts
  return preds, acts

In [27]:
train(model, train_loader, val_loader, optimizer)

0 352
100 352
200 352
300 352
Epoch 1: train_loss: 0.4962 train_acc: 0.7625 | val_loss: 0.3994 val_acc: 0.8338
00:10:06.23
0 352
100 352
200 352
300 352
Epoch 2: train_loss: 0.3808 train_acc: 0.8397 | val_loss: 0.3962 val_acc: 0.8263
00:10:06.98
0 352
100 352
200 352
300 352
Epoch 3: train_loss: 0.2972 train_acc: 0.8819 | val_loss: 0.4250 val_acc: 0.8320
00:10:07.11
0 352
100 352
200 352
300 352
Epoch 4: train_loss: 0.2133 train_acc: 0.9191 | val_loss: 0.4321 val_acc: 0.8444
00:10:06.42
0 352
100 352
200 352
300 352
Epoch 5: train_loss: 0.1549 train_acc: 0.9438 | val_loss: 0.4614 val_acc: 0.8419
00:10:06.74


In [39]:
preds, acts = test(model, test_loader)

test_loss: 0.4463 test_acc: 0.8420
00:00:24.93


In [44]:
preds_a, acts_a = test(model, test_loader_a)

test_loss: 0.5944 test_acc: 0.7969
00:00:10.40


In [50]:
#CLASS A:
tp_a = 0
fp_a = 0
tn_a = 0
fn_a = 0
preds_a = [math.exp(preds_a[i][1])/(math.exp(preds_a[i][0]) + math.exp(preds_a[i][1])) for i in range(len(preds_a))]
for i in range(len(preds_a)):
    if acts_a[i] == 1:
        if preds_a[i] >= 0.5:
            tp_a += 1
        else:
            fn_a += 1
    else:
        if preds_a[i] < 0.5:
            tn_a += 1
        else:
            fp_a += 1
accuracy_a = (tp_a + tn_a)/(tp_a + fp_a + fn_a + tn_a)
recall_a = tp_a/(tp_a + fn_a)
precision_a = tp_a/(tp_a + fp_a)
f1_a = (2*precision_a*recall_a)/(precision_a + recall_a)
print("TP: {}, FP: {}\nFN: {}, TN: {}".format(tp_a, fp_a, fn_a, tn_a))
print("Recall:", recall_a)
print("Precsion:", precision_a)
print("F1:", f1_a)
print("Accuracy", accuracy_a)

TP: 288, FP: 132
FN: 124, TN: 718
Recall: 0.6990291262135923
Precsion: 0.6857142857142857
F1: 0.6923076923076924
Accuracy 0.7971473851030111


In [51]:
ent_acts_a = 0
cont_acts_a = 0
correct_ents_a = 0
correct_conts_a = 0
for i in range(len(preds_a)):
    if acts_a[i] == 1:
        ent_acts_a += 1
        if preds_a[i] >= 0.5:
            correct_ents_a += 1
    else:
        cont_acts_a += 1
        if preds_a[i] < 0.5:
            correct_conts_a += 1
            
print("Total offensive examples:", ent_acts_a)
print("Correctly predicted offensive examples:", correct_ents_a)
print("Accuracy for predicting offensive examples:", correct_ents_a/ent_acts_a)
print("Total non offensive examples:", cont_acts_a)
print("Correctly predicted non offensive examples:", correct_conts_a)
print("Accuracy for predicting non offensive examples:", correct_conts_a/cont_acts_a)

Total offensive examples: 412
Correctly predicted offensive examples: 288
Accuracy for predicting offensive examples: 0.6990291262135923
Total non offensive examples: 850
Correctly predicted non offensive examples: 718
Accuracy for predicting non offensive examples: 0.8447058823529412


In [52]:
preds_b, acts_b = test(model, test_loader_b)

test_loss: 0.3126 test_acc: 0.8971
00:00:02.51


In [53]:
#CLASS B:
tp_b = 0
fp_b = 0
tn_b = 0
fn_b = 0
preds_b = [math.exp(preds_b[i][1])/(math.exp(preds_b[i][0]) + math.exp(preds_b[i][1])) for i in range(len(preds_b))]
for i in range(len(preds_b)):
    if acts_b[i] == 1:
        if preds_b[i] >= 0.5:
            tp_b += 1
        else:
            fn_b += 1
    else:
        if preds_b[i] < 0.5:
            tn_b += 1
        else:
            fp_b += 1
accuracy_b = (tp_b + tn_b)/(tp_b + fp_b + fn_b + tn_b)
recall_b = tp_b/(tp_b + fn_b)
precision_b = tp_b/(tp_b + fp_b)
f1_b = (2*precision_b*recall_b)/(precision_b + recall_b)
print("TP: {}, FP: {}\nFN: {}, TN: {}".format(tp_b, fp_b, fn_b, tn_b))
print("Recall:", recall_b)
print("Precsion:", precision_b)
print("F1:", f1_b)
print("Accuracy", accuracy_b)

TP: 376, FP: 32
FN: 13, TN: 10
Recall: 0.9665809768637532
Precsion: 0.9215686274509803
F1: 0.9435382685069008
Accuracy 0.8955916473317865


In [54]:
ent_acts_b = 0
cont_acts_b = 0
correct_ents_b = 0
correct_conts_b = 0
for i in range(len(preds_b)):
    if acts_b[i] == 1:
        ent_acts_b += 1
        if preds_b[i] > 0.5:
            correct_ents_b += 1
    else:
        cont_acts_b += 1
        if preds_b[i] < 0.5:
            correct_conts_b += 1
            
print("Total targeted offensive examples:", ent_acts_b)
print("Correctly predicted targeted offensive examples:", correct_ents_b)
print("Accuracy for predicting targeted offensive examples:", correct_ents_b/ent_acts_b)
print("Total non targeted offensive examples:", cont_acts_b)
print("Correctly predicted non targeted offensive examples:", correct_conts_b)
print("Accuracy for predicting non targeted offensive examples:", correct_conts_b/cont_acts_b)

Total targeted offensive examples: 389
Correctly predicted targeted offensive examples: 376
Accuracy for predicting targeted offensive examples: 0.9665809768637532
Total non targeted offensive examples: 42
Correctly predicted non targeted offensive examples: 10
Accuracy for predicting non targeted offensive examples: 0.23809523809523808


In [58]:
preds_c, acts_c = test(model, test_loader_c)

test_loss: 0.3601 test_acc: 0.8747
00:00:03.56


In [59]:
#CLASS C:
tp_c = 0
fp_c = 0
tn_c = 0
fn_c = 0
preds_c = [math.exp(preds_c[i][1])/(math.exp(preds_c[i][0]) + math.exp(preds_c[i][1])) for i in range(len(preds_c))]
for i in range(len(preds_c)):
    if acts_c[i] == 1:
        if preds_c[i] >= 0.5:
            tp_c += 1
        else:
            fn_c += 1
    else:
        if preds_c[i] < 0.5:
            tn_c += 1
        else:
            fp_c += 1
accuracy_c = (tp_c + tn_c)/(tp_c + fp_c + fn_c + tn_c)
recall_c = tp_c/(tp_c + fn_c)
precision_c = tp_c/(tp_c + fp_c)
f1_c = (2*precision_c*recall_c)/(precision_c + recall_c)
print("TP: {}, FP: {}\nFN: {}, TN: {}".format(tp_c, fp_c, fn_c, tn_c))
print("Recall:", recall_c)
print("Precsion:", precision_c)
print("F1:", f1_c)
print("Accuracy", accuracy_c)

TP: 210, FP: 19
FN: 30, TN: 139
Recall: 0.875
Precsion: 0.9170305676855895
F1: 0.8955223880597014
Accuracy 0.8768844221105527


In [41]:
ent_acts_c = 0
cont_acts_c = 0
correct_ents_c = 0
correct_conts_c = 0
for i in range(len(preds_c)):
    if acts_c[i] == 1:
        ent_acts_c += 1
        if preds_c[i] >= 0.5:
            correct_ents_c += 1
    else:
        cont_acts_c += 1
        if preds_c[i] < 1:
            correct_conts_c += 1
            
print("Total individual targeted offensive examples:", ent_acts_c)
print("Correctly predicted individual targeted offensive examples:", correct_ents_c)
print("Accuracy for predicting individual targeted offensive examples:", correct_ents_c/ent_acts_c)
print("Total non individual targeted offensive examples:", cont_acts_c)
print("Correctly predicted non individual targeted offensive examples:", correct_conts_c)
print("Accuracy for predicting non individual targeted offensive examples:", correct_conts_c/cont_acts_c)

Total individual targeted offensive examples: 224
Correctly predicted individual targeted offensive examples: 198
Accuracy for predicting individual targeted offensive examples: 0.8839285714285714
Total non individual targeted offensive examples: 135
Correctly predicted non individual targeted offensive examples: 114
Accuracy for predicting non individual targeted offensive examples: 0.8444444444444444


In [61]:
preds_d, acts_d = test(model, test_loader_d)

test_loss: 0.4307 test_acc: 0.8297
00:00:01.63


In [62]:
#CLASS D:
tp_d = 0
fp_d = 0
tn_d = 0
fn_d = 0
preds_d = [math.exp(preds_d[i][1])/(math.exp(preds_d[i][0]) + math.exp(preds_d[i][1])) for i in range(len(preds_d))]
for i in range(len(preds_d)):
    if acts_d[i] == 1:
        if preds_d[i] >= 0.5:
            tp_d += 1
        else:
            fn_d += 1
    else:
        if preds_d[i] < 0.5:
            tn_d += 1
        else:
            fp_d += 1
accuracy_d = (tp_d + tn_d)/(tp_d + fp_d + fn_d + tn_d)
recall_d = tp_d/(tp_d + fn_d)
precision_d = tp_d/(tp_d + fp_d)
f1_d = (2*precision_d*recall_d)/(precision_d + recall_d)
print("TP: {}, FP: {}\nFN: {}, TN: {}".format(tp_d, fp_d, fn_d, tn_d))
print("Recall:", recall_d)
print("Precsion:", precision_d)
print("F1:", f1_d)
print("Accuracy", accuracy_d)

TP: 82, FP: 40
FN: 21, TN: 220
Recall: 0.7961165048543689
Precsion: 0.6721311475409836
F1: 0.7288888888888888
Accuracy 0.8319559228650137


In [63]:
ent_acts_d = 0
cont_acts_d = 0
correct_ents_d = 0
correct_conts_d = 0
for i in range(len(preds_d)):
    if acts_d[i] == 1:
        ent_acts_d += 1
        if preds_d[i] >= 0.5:
            correct_ents_d += 1
    else:
        cont_acts_d += 1
        if preds_d[i] < 0.5:
            correct_conts_d += 1
            
print("Total group targeted offensive examples:", ent_acts_d)
print("Correctly predicted group targeted offensive examples:", correct_ents_d)
print("Accuracy for predicting group targeted offensive examples:", correct_ents_d/ent_acts_d)
print("Total non group targeted offensive examples:", cont_acts_d)
print("Correctly predicted non group targeted offensive examples:", correct_conts_d)
print("Accuracy for predicting non group targeted offensive examples:", correct_conts_d/cont_acts_d)

Total group targeted offensive examples: 103
Correctly predicted group targeted offensive examples: 82
Accuracy for predicting group targeted offensive examples: 0.7961165048543689
Total non group targeted offensive examples: 260
Correctly predicted non group targeted offensive examples: 220
Accuracy for predicting non group targeted offensive examples: 0.8461538461538461


In [64]:
preds_e, acts_e = test(model, test_loader_e)

test_loss: 0.2136 test_acc: 0.9060
00:00:02.36


In [65]:
#CLASS E:
tp_e = 0
fp_e = 0
tn_e = 0
fn_e = 0
preds_e = [math.exp(preds_e[i][1])/(math.exp(preds_e[i][0]) + math.exp(preds_e[i][1])) for i in range(len(preds_e))]
for i in range(len(preds_e)):
    if acts_e[i] == 1:
        if preds_e[i] >= 0.5:
            tp_e += 1
        else:
            fn_e += 1
    else:
        if preds_e[i] < 0.5:
            tn_e += 1
        else:
            fp_e += 1
accuracy_e = (tp_e + tn_e)/(tp_e + fp_e + fn_e + tn_e)
recall_e = tp_e/(tp_e + fn_e)
precision_e = tp_e/(tp_e + fp_e)
f1_e = (2*precision_e*recall_e)/(precision_e + recall_e)
print("TP: {}, FP: {}\nFN: {}, TN: {}".format(tp_e, fp_e, fn_e, tn_e))
print("Recall:", recall_e)
print("Precsion:", precision_e)
print("F1:", f1_e)
print("Accuracy", accuracy_e)

TP: 2, FP: 4
FN: 30, TN: 326
Recall: 0.0625
Precsion: 0.3333333333333333
F1: 0.10526315789473684
Accuracy 0.9060773480662984


In [66]:
ent_acts_e = 0
cont_acts_e = 0
correct_ents_e = 0
correct_conts_e = 0
for i in range(len(preds_e)):
    if acts_e[i] == 1:
        ent_acts_e += 1
        if preds_e[i] >= 0.5:
            correct_ents_e += 1
    else:
        cont_acts_e += 1
        if preds_e[i] < 0.5:
            correct_conts_e += 1
            
print("Total other targeted offensive examples:", ent_acts_e)
print("Correctly predicted other targeted offensive examples:", correct_ents_e)
print("Accuracy for predicting other targeted offensive examples:", correct_ents_e/ent_acts_e)
print("Total non other targeted offensive examples:", cont_acts_e)
print("Correctly predicted non other targeted offensive examples:", correct_conts_e)
print("Accuracy for predicting non other targeted offensive examples:", correct_conts_e/cont_acts_e)

Total other targeted offensive examples: 32
Correctly predicted other targeted offensive examples: 2
Accuracy for predicting other targeted offensive examples: 0.0625
Total non other targeted offensive examples: 330
Correctly predicted non other targeted offensive examples: 326
Accuracy for predicting non other targeted offensive examples: 0.9878787878787879


In [82]:
f1_3 = (f1_c + f1_d + f1_e)/3
print("Macro average F1 for class 3 IND/GRP/OTH is", f1_3)
print("Macro average F1 overall", (f1_a + f1_b + f1_3)/3)

Macro average F1 for class 3 IND/GRP/OTH is 0.5765581449477756
Macro average F1 overall 0.737468035254123


In [73]:
def test_transfer(model, test_loader):
  #model.load_state_dict(torch.load('WeightsOffenseval/nli_latest_3.pt', map_location=torch.device('cpu')))
  model.load_state_dict(torch.load('WeightsHateval/nli_new_4.pt', map_location=torch.device('cpu')))
  model.eval()
  total_test_acc  = 0
  total_test_loss = 0
  start = time.time()
  preds = []
  acts = []
  with torch.no_grad():
    #print("Prediction", "Label")
    for batch_idx, (pair_token_ids, mask_ids, seg_ids, y) in enumerate(test_loader):
      optimizer.zero_grad()
      pair_token_ids = pair_token_ids.to(device)
      mask_ids = mask_ids.to(device)
      seg_ids = seg_ids.to(device)
      labels = y.to(device)

      # prediction = model(pair_token_ids, mask_ids, seg_ids)
      loss, prediction = model(pair_token_ids, 
                           token_type_ids=seg_ids, 
                           attention_mask=mask_ids, 
                           labels=labels).values()
        
      # loss = criterion(prediction, labels)
      acc = multi_acc(prediction, labels)
      preds = preds + prediction.tolist()
      acts = acts + labels.tolist()
      #print(prediction, labels)
      total_test_loss += loss.item()
      total_test_acc  += acc.item() 

  test_acc  = total_test_acc/len(test_loader)
  test_loss = total_test_loss/len(test_loader)
  end = time.time()
  hours, rem = divmod(end-start, 3600)
  minutes, seconds = divmod(rem, 60)
  print(f'test_loss: {test_loss:.4f} test_acc: {test_acc:.4f}')
  print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
  #return [math.exp(preds[i][1])/(math.exp(preds[i][0] + math.exp(preds[i][1]))) for i in range(len(preds))], acts
  #return [0 if preds[i][0] > preds[i][1] else 1 for i in range(len(preds))], acts
  return preds, acts

In [74]:
preds_a_transfer, acts_a_transfer = test_transfer(model, test_loader_a)

test_loss: 1.6622 test_acc: 0.6454
00:00:10.31


In [75]:
preds_b_transfer, acts_b_transfer = test_transfer(model, test_loader_b)

test_loss: 2.5022 test_acc: 0.4186
00:00:02.48


In [76]:
preds_c_transfer, acts_c_transfer = test_transfer(model, test_loader_c)

test_loss: 1.9492 test_acc: 0.5555
00:00:03.54


In [77]:
preds_d_transfer, acts_d_transfer = test_transfer(model, test_loader_d)

test_loss: 2.1514 test_acc: 0.4551
00:00:01.65


In [78]:
preds_e_transfer, acts_e_transfer = test_transfer(model, test_loader_e)

test_loss: 1.1987 test_acc: 0.6226
00:00:02.34


In [79]:
#CLASS A: Transfer learning from hateval

tp_a_transfer = 0
fp_a_transfer = 0
tn_a_transfer = 0
fn_a_transfer = 0
preds_a_transfer = [math.exp(preds_a_transfer[i][1])/(math.exp(preds_a_transfer[i][0]) + math.exp(preds_a_transfer[i][1])) for i in range(len(preds_e))]
for i in range(len(preds_a_transfer)):
    if acts_a_transfer[i] == 1:
        if preds_a_transfer[i] >= 0.5:
            tp_a_transfer += 1
        else:
            fn_a_transfer += 1
    else:
        if preds_a_transfer[i] < 0.5:
            tn_a_transfer += 1
        else:
            fp_a_transfer += 1
accuracy_a_transfer = (tp_a_transfer + tn_a_transfer)/(tp_a_transfer + fp_a_transfer + fn_a_transfer + tn_a_transfer)
recall_a_transfer = tp_a_transfer/(tp_a_transfer + fn_a_transfer)
precision_a_transfer = tp_a_transfer/(tp_a_transfer + fp_a_transfer)
f1_a_transfer = (2*precision_a_transfer*recall_a_transfer)/(precision_a_transfer + recall_a_transfer)
print("TP: {}, FP: {}\nFN: {}, TN: {}".format(tp_a_transfer, fp_a_transfer, fn_a_transfer, tn_a_transfer))
print("Recall:", recall_a_transfer)
print("Precsion:", precision_a_transfer)
print("F1:", f1_a_transfer)
print("Accuracy", accuracy_a_transfer)

TP: 32, FP: 29
FN: 90, TN: 211
Recall: 0.26229508196721313
Precsion: 0.5245901639344263
F1: 0.3497267759562842
Accuracy 0.6712707182320442


In [80]:
#CLASS B: Transfer learning from hateval

tp_b_transfer = 0
fp_b_transfer = 0
tn_b_transfer = 0
fn_b_transfer = 0
preds_b_transfer = [math.exp(preds_b_transfer[i][1])/(math.exp(preds_b_transfer[i][0]) + math.exp(preds_b_transfer[i][1])) for i in range(len(preds_e))]
for i in range(len(preds_b_transfer)):
    if acts_b_transfer[i] == 1:
        if preds_b_transfer[i] >= 0.5:
            tp_b_transfer += 1
        else:
            fn_b_transfer += 1
    else:
        if preds_b_transfer[i] < 0.5:
            tn_b_transfer += 1
        else:
            fp_b_transfer += 1
accuracy_b_transfer = (tp_b_transfer + tn_b_transfer)/(tp_b_transfer + fp_b_transfer + fn_b_transfer + tn_b_transfer)
recall_b_transfer = tp_b_transfer/(tp_b_transfer + fn_b_transfer)
precision_b_transfer = tp_b_transfer/(tp_b_transfer + fp_b_transfer)
f1_b_transfer = (2*precision_b_transfer*recall_b_transfer)/(precision_b_transfer + recall_b_transfer)
print("TP: {}, FP: {}\nFN: {}, TN: {}".format(tp_b_transfer, fp_b_transfer, fn_b_transfer, tn_b_transfer))
print("Recall:", recall_b_transfer)
print("Precsion:", precision_b_transfer)
print("F1:", f1_b_transfer)
print("Accuracy", accuracy_b_transfer)

TP: 130, FP: 8
FN: 198, TN: 26
Recall: 0.39634146341463417
Precsion: 0.9420289855072463
F1: 0.5579399141630902
Accuracy 0.430939226519337


In [81]:
#CLASS C: Transfer learning from hateval

tp_c_transfer = 0
fp_c_transfer = 0
tn_c_transfer = 0
fn_c_transfer = 0
preds_c_transfer = [math.exp(preds_c_transfer[i][1])/(math.exp(preds_c_transfer[i][0]) + math.exp(preds_c_transfer[i][1])) for i in range(len(preds_e))]
for i in range(len(preds_c_transfer)):
    if acts_c_transfer[i] == 1:
        if preds_c_transfer[i] >= 0.5:
            tp_c_transfer += 1
        else:
            fn_c_transfer += 1
    else:
        if preds_c_transfer[i] < 0.5:
            tn_c_transfer += 1
        else:
            fp_c_transfer += 1
accuracy_c_transfer = (tp_c_transfer + tn_c_transfer)/(tp_c_transfer + fp_c_transfer + fn_c_transfer + tn_c_transfer)
recall_c_transfer = tp_c_transfer/(tp_c_transfer + fn_c_transfer)
precision_c_transfer = tp_c_transfer/(tp_c_transfer + fp_c_transfer)
f1_c_transfer = (2*precision_c_transfer*recall_c_transfer)/(precision_c_transfer + recall_c_transfer)
print("TP: {}, FP: {}\nFN: {}, TN: {}".format(tp_c_transfer, fp_c_transfer, fn_c_transfer, tn_c_transfer))
print("Recall:", recall_c_transfer)
print("Precsion:", precision_c_transfer)
print("F1:", f1_c_transfer)
print("Accuracy", accuracy_c_transfer)

TP: 96, FP: 23
FN: 127, TN: 116
Recall: 0.4304932735426009
Precsion: 0.8067226890756303
F1: 0.5614035087719298
Accuracy 0.585635359116022


In [83]:
#CLASS D: Transfer learning from hateval

tp_d_transfer = 0
fp_d_transfer = 0
tn_d_transfer = 0
fn_d_transfer = 0
preds_d_transfer = [math.exp(preds_d_transfer[i][1])/(math.exp(preds_d_transfer[i][0]) + math.exp(preds_d_transfer[i][1])) for i in range(len(preds_e))]
for i in range(len(preds_d_transfer)):
    if acts_d_transfer[i] == 1:
        if preds_d_transfer[i] >= 0.5:
            tp_d_transfer += 1
        else:
            fn_d_transfer += 1
    else:
        if preds_d_transfer[i] < 0.5:
            tn_d_transfer += 1
        else:
            fp_d_transfer += 1
accuracy_d_transfer = (tp_d_transfer + tn_d_transfer)/(tp_d_transfer + fp_d_transfer + fn_d_transfer + tn_d_transfer)
recall_d_transfer = tp_d_transfer/(tp_d_transfer + fn_d_transfer)
precision_d_transfer = tp_d_transfer/(tp_d_transfer + fp_d_transfer)
f1_d_transfer = (2*precision_d_transfer*recall_d_transfer)/(precision_d_transfer + recall_d_transfer)
print("TP: {}, FP: {}\nFN: {}, TN: {}".format(tp_d_transfer, fp_d_transfer, fn_d_transfer, tn_d_transfer))
print("Recall:", recall_d_transfer)
print("Precsion:", precision_d_transfer)
print("F1:", f1_d_transfer)
print("Accuracy", accuracy_d_transfer)

TP: 12, FP: 107
FN: 91, TN: 152
Recall: 0.11650485436893204
Precsion: 0.10084033613445378
F1: 0.10810810810810811
Accuracy 0.4530386740331492


In [84]:
#CLASS D: Transfer learning from hateval

tp_e_transfer = 0
fp_e_transfer = 0
tn_e_transfer = 0
fn_e_transfer = 0
preds_e_transfer = [math.exp(preds_e_transfer[i][1])/(math.exp(preds_e_transfer[i][0]) + math.exp(preds_e_transfer[i][1])) for i in range(len(preds_e))]
for i in range(len(preds_e_transfer)):
    if acts_e_transfer[i] == 1:
        if preds_e_transfer[i] >= 0.5:
            tp_e_transfer += 1
        else:
            fn_e_transfer += 1
    else:
        if preds_e_transfer[i] < 0.5:
            tn_e_transfer += 1
        else:
            fp_e_transfer += 1
accuracy_e_transfer = (tp_e_transfer + tn_e_transfer)/(tp_e_transfer + fp_e_transfer + fn_e_transfer + tn_e_transfer)
recall_e_transfer = tp_e_transfer/(tp_e_transfer + fn_e_transfer)
precision_e_transfer = tp_e_transfer/(tp_e_transfer + fp_e_transfer)
f1_e_transfer = (2*precision_e_transfer*recall_e_transfer)/(precision_e_transfer + recall_e_transfer)
print("TP: {}, FP: {}\nFN: {}, TN: {}".format(tp_e_transfer, fp_e_transfer, fn_e_transfer, tn_e_transfer))
print("Recall:", recall_e_transfer)
print("Precsion:", precision_e_transfer)
print("F1:", f1_e_transfer)
print("Accuracy", accuracy_e_transfer)

TP: 3, FP: 107
FN: 29, TN: 223
Recall: 0.09375
Precsion: 0.02727272727272727
F1: 0.04225352112676056
Accuracy 0.6243093922651933


In [85]:
#overall results for transfer learning from hateval

f1_3_transfer = (f1_c_transfer + f1_d_transfer + f1_e_transfer)/3
print("Macro average F1 for class 3 IND/GRP/OTH is", f1_3_transfer)
print("Macro average F1 overall", (f1_a_transfer + f1_b_transfer + f1_3_transfer)/3)

Macro average F1 for class 3 IND/GRP/OTH is 0.23725504600226613
Macro average F1 overall 0.3816405787072135
